In [1]:
# This is a notebook to summarize all different models.
# THis notebook was not run


## Balanced AutoEncoder
# encoder_dense_layers_trial = [[10, 8], [12, 10], [14, 12], [16, 14], [18, 16], [20, 18] ,[22,20]]
# decoder_dense_layers_trial = [[8, 10], [10, 12], [12, 14], [14, 16], [16, 18], [18, 20], [20,22]]
# bottle_neck_trial = [8, 10, 12, 14, 16, 18]

In [2]:
#Disable Warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

In [3]:
FILE1 = 'SB-001_normalized_AM.csv'
FILE2 = 'SB-001_normalized_PM.csv'
NO_OF_COLUMNS = 24

In [4]:
#pip install tf2onnx
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
plt.ioff()
import cv2
import IPython
from tqdm import tqdm
import time
import os
import pickle
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', message=r'.*Consider either turning off auto-sharding.*')

In [5]:
run_summary = []

In [6]:
def generate_autoencoder_reports(encoder_dense_layers, bottle_neck, decoder_dense_layers):

    print(encoder_dense_layers, bottle_neck, decoder_dense_layers)
    # Load Data
    df_am = pd.read_csv(FILE1)
    df_pm = pd.read_csv(FILE2)
    combined_df = pd.concat([df_am, df_pm])

    columns_needed = ['y_am_pef', 'tempin', 'humidin', 'pm25in', 'co2in', 'tempdiffin', 'humidiffin', 'pm25diffin',
                      'pm10', 'pm25', 'o3', 'no2', 'co', 'so2', 'temp', 'windsd', 'humid', 'varp', 'dewpt', 'airp',
                      'seap', 'solrhr', 'solramnt', 'grdt', 'class']
    combined_df = combined_df.filter(columns_needed)
    df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Drop NaN
    df = df.dropna()

    # Minotity Split
    class_counts = df['class'].value_counts()
    minority_class = df[df['class'] == 0]
    majority_class = df[df['class'] == 1]

    # Removing class column from minority before augmentation
    X = minority_class.drop('class', axis=1)
    y = minority_class['class']

    # Saving minority as X_train
    X_train = X

    # Define the Input shape
    INPUT_SHAPE = X_train.shape[1]
    FILE_NAME = f"L{INPUT_SHAPE}_E{'_'.join(map(str, encoder_dense_layers))}_B{bottle_neck}_D{'_'.join(map(str, decoder_dense_layers))}"
    print("\n" + FILE_NAME + "\n")
    
    def build_autoencoder(input_shape, **kwargs):

        # Autoencoder model construction cod

      """
      Build an autoencoder model with the given configuration.

      Args:
          input_shape (int): The shape of the input data.
          **kwargs: Additional keyword arguments for configuring the autoencoder.

      Keyword Args:
          encoder_dense_layers (list): List of integers specifying the number of units for each dense layer in the encoder.
                                      Default: []
          bottle_neck (int): The number of units in the bottleneck layer. Default: input_shape // 2
          decoder_dense_layers (list): List of integers specifying the number of units for each dense layer in the decoder.
                                      Default: []
          decoder_activation (str): The activation function for the decoder output layer. Default: 'sigmoid'

      Returns:
          tuple: A tuple containing the autoencoder, encoder, and decoder models.

      Example:
          INPUT_SHAPE = 27
          encoder_dense_layers = [20, 18]  # Specify the number of units for each dense layer in the encoder
          bottle_neck = 16
          decoder_dense_layers = [18, 20]  # Specify the number of units for each dense layer in the decoder

          autoencoder, encoder, decoder = build_autoencoder(INPUT_SHAPE, encoder_dense_layers=encoder_dense_layers,
                                                            bottle_neck=bottle_neck, decoder_dense_layers=decoder_dense_layers)
          encoder.summary()
          decoder.summary()
          autoencoder.summary()
      """

      # Default parameter values
      encoder_dense_layers = kwargs.get('encoder_dense_layers', [])
      bottle_neck = kwargs.get('bottle_neck', input_shape // 2)
      decoder_dense_layers = kwargs.get('decoder_dense_layers', [])
      decoder_activation = kwargs.get('decoder_activation', 'sigmoid')

      # Autoencoder Model
      encoder_input = keras.Input(shape=(input_shape,), name="encoder")
      x = keras.layers.Flatten()(encoder_input)

      # Encoder Dense Layers
      for units in encoder_dense_layers:
          x = keras.layers.Dense(units, activation="relu")(x)

      encoder_output = keras.layers.Dense(bottle_neck, activation="relu")(x)
      encoder = keras.Model(encoder_input, encoder_output, name="encoder")

      # Decoder Model
      decoder_input = keras.Input(shape=(bottle_neck,), name="decoder")
      x = decoder_input

      # Decoder Dense Layers
      for units in decoder_dense_layers:
          x = keras.layers.Dense(units, activation="relu")(x)

      decoder_output = keras.layers.Dense(input_shape, activation=decoder_activation)(x)
      decoder = keras.Model(decoder_input, decoder_output, name="decoder")

      # Autoencoder Model
      autoencoder_input = keras.Input(shape=(input_shape,), name="input")
      encoded = encoder(autoencoder_input)
      decoded = decoder(encoded)
      autoencoder = keras.Model(autoencoder_input, decoded, name="autoencoder")

      return autoencoder, encoder, decoder

    autoencoder, encoder, decoder = build_autoencoder(INPUT_SHAPE, encoder_dense_layers=encoder_dense_layers,
                                                      bottle_neck=bottle_neck,
                                                      decoder_dense_layers=decoder_dense_layers)
    opt = keras.optimizers.Adam(learning_rate=0.001)
    autoencoder.compile(opt, loss="mse")

    print("Training model:", FILE_NAME)
    history = autoencoder.fit(X_train, X_train, epochs=200, batch_size=16, validation_split=0.25, verbose=0)
    print("Training Complete:")
    
    # Extract the loss values
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Print the last epoch's loss values
    last_epoch_loss = loss[-1]
    last_epoch_val_loss = val_loss[-1]
    print("Last epoch loss:", last_epoch_loss)
    print("Last epoch validation loss:", last_epoch_val_loss)

    # Saving history
    with open(FILE_NAME + '_history.pickle', 'wb') as file:
        pickle.dump(history.history, file)

    # Visualize losses *(MSE)
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(FILE_NAME + 'Loss.png')
    #plt.show()

    # Generate synthetic data
    num_samples = len(X_train)
    input_data = np.random.normal(size=(num_samples, INPUT_SHAPE))
    generated_data = autoencoder.predict(input_data)
    reshaped_data = generated_data.reshape(num_samples, -1)
    df_generated = pd.DataFrame(reshaped_data, columns=X_train.columns)

    # Plot correlation matrices
    corr_matrix1 = X_train.corr()
    corr_matrix2 = df_generated.corr()

    fig, axes = plt.subplots(1, 2, figsize=(18, 10))
    sns.heatmap(corr_matrix1, annot=True, cmap='coolwarm', ax=axes[0])
    axes[0].set_title('Heatmap 1')

    sns.heatmap(corr_matrix2, annot=True, cmap='coolwarm', ax=axes[1])
    axes[1].set_title('Heatmap 2')

    plt.tight_layout()
    plt.savefig(FILE_NAME + 'HeatMaps')
    #plt.show()

    # Calculate mean and standard deviation of original and synthetic datasets
    common_columns = set(X_train.columns) & set(df_generated.columns)
    results = {}

    for column in common_columns:
        mean_df1 = X_train[column].mean()
        std_df1 = X_train[column].std()
        mean_df2 = df_generated[column].mean()
        std_df2 = df_generated[column].std()

        results[column] = {'Mean_df1': mean_df1, 'Std_df1': std_df1,
                           'Mean_df2': mean_df2, 'Std_df2': std_df2}

    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv(FILE_NAME + 'mean_std.csv', index=True)

    # Plot scatter plots
    new_index = pd.RangeIndex(start=0, stop=57, step=1)
    X_train = X_train.set_index(new_index)
    common_columns = set(X_train.columns) & set(df_generated.columns)

    for column in common_columns:
        plt.scatter(X_train.index, X_train[column], color='red', label='X_train')
        plt.scatter(df_generated.index, df_generated[column], color='blue', label='df_generated')

        plt.title(f"Scatter Plot: {column}")
        plt.xlabel("Index")
        plt.ylabel(column)

        plt.legend()
        plt.savefig(FILE_NAME + "_" + column)
        #plt.show()


    # Add back the class label
    X_train['class'] = 0.0
    df_generated['class'] = 0.0
    X_train.to_csv(FILE_NAME + 'Original_minority_data.csv', index=False)
    df_generated.to_csv(FILE_NAME + 'Synthetic_minority_data.csv', index=False)

    # Generate quality report and display it
    real_data = FILE_NAME + 'Original_minority_data.csv'
    synth_data = FILE_NAME + 'Synthetic_minority_data.csv'

    # report = QualityReport(data_source=synth_data, ref_data=real_data)
    # report.run()

    # print(report.peek())
    #run_summary.append([FILE_NAME, report.peek()['raw_score'], report.peek()['grade']])
    run_summary.append([FILE_NAME, last_epoch_loss, last_epoch_val_loss])

    print("Reports Saved")
    #IPython.display.HTML(report.as_html, metadata=dict(isolated=True))
    return None

In [7]:
# Example usage
def runner(encoder_dense_layers, bottle_neck, decoder_dense_layers):
  #encoder_dense_layers = [10, 8]
  #bottle_neck = 5
  #decoder_dense_layers = [8, 10]
  #FILE_NAME = f"24_{'_'.join(map(str, encoder_dense_layers))}_{bottle_neck}_"
  FILE_NAME = f"L{NO_OF_COLUMNS}_E{'_'.join(map(str, encoder_dense_layers))}_B{bottle_neck}_D{'_'.join(map(str, decoder_dense_layers))}"   
  report = generate_autoencoder_reports(encoder_dense_layers, bottle_neck, decoder_dense_layers)
  #print(type(report))
  # Save the report object to a file
  # with open(FILE_NAME + 'quality_report.pickle', 'wb') as file:
  #     pickle.dump(report, file)

  #IPython.display.HTML(report.as_html, metadata=dict(isolated=True))

In [8]:
# here goes nothin

In [9]:
encoder_dense_layers_trial = [[10, 8], [12, 10], [14, 12], [16, 14], [18, 16], [20, 18] ,[22,20]]
decoder_dense_layers_trial = [[8, 10], [10, 12], [12, 14], [14, 16], [16, 18], [18, 20], [20,22]]
bottle_neck_trial = [8, 10, 12, 14, 16, 18]

In [10]:
total_iterations = len(bottle_neck_trial) * len(encoder_dense_layers_trial) * len(decoder_dense_layers_trial)
print("Total Model in Pipeline:",total_iterations)

# Print total models
for bn in bottle_neck_trial:
  for enc_layers in encoder_dense_layers_trial:
      for dec_layers in decoder_dense_layers_trial:
          print(enc_layers, bn, dec_layers)

Total Model in Pipeline: 294
[10, 8] 8 [8, 10]
[10, 8] 8 [10, 12]
[10, 8] 8 [12, 14]
[10, 8] 8 [14, 16]
[10, 8] 8 [16, 18]
[10, 8] 8 [18, 20]
[10, 8] 8 [20, 22]
[12, 10] 8 [8, 10]
[12, 10] 8 [10, 12]
[12, 10] 8 [12, 14]
[12, 10] 8 [14, 16]
[12, 10] 8 [16, 18]
[12, 10] 8 [18, 20]
[12, 10] 8 [20, 22]
[14, 12] 8 [8, 10]
[14, 12] 8 [10, 12]
[14, 12] 8 [12, 14]
[14, 12] 8 [14, 16]
[14, 12] 8 [16, 18]
[14, 12] 8 [18, 20]
[14, 12] 8 [20, 22]
[16, 14] 8 [8, 10]
[16, 14] 8 [10, 12]
[16, 14] 8 [12, 14]
[16, 14] 8 [14, 16]
[16, 14] 8 [16, 18]
[16, 14] 8 [18, 20]
[16, 14] 8 [20, 22]
[18, 16] 8 [8, 10]
[18, 16] 8 [10, 12]
[18, 16] 8 [12, 14]
[18, 16] 8 [14, 16]
[18, 16] 8 [16, 18]
[18, 16] 8 [18, 20]
[18, 16] 8 [20, 22]
[20, 18] 8 [8, 10]
[20, 18] 8 [10, 12]
[20, 18] 8 [12, 14]
[20, 18] 8 [14, 16]
[20, 18] 8 [16, 18]
[20, 18] 8 [18, 20]
[20, 18] 8 [20, 22]
[22, 20] 8 [8, 10]
[22, 20] 8 [10, 12]
[22, 20] 8 [12, 14]
[22, 20] 8 [14, 16]
[22, 20] 8 [16, 18]
[22, 20] 8 [18, 20]
[22, 20] 8 [20, 22]
[10, 

In [ ]:
'''
Type	Bottleneck	Model	Training Loss	Validation Loss
Balanced	B8	L24_E10_8_B8_D8_10	0.014485765	0.022974495
Balanced	B8	L24_E10_8_B8_D10_12	0.01060681	0.017263062
Balanced	B8	L24_E10_8_B8_D12_14	0.011916933	0.018185237
Balanced	B8	L24_E10_8_B8_D14_16	0.011970292	0.019739432
Balanced	B8	L24_E10_8_B8_D16_18	0.019925587	0.030561503
Balanced	B8	L24_E10_8_B8_D18_20	0.010648762	0.017230611
Balanced	B8	L24_E10_8_B8_D20_22	0.008817361	0.014682335
Balanced	B8	L24_E12_10_B8_D8_10	0.019411495	0.029249368
Balanced	B8	L24_E12_10_B8_D10_12	0.01750912	0.026244964
Balanced	B8	L24_E12_10_B8_D12_14	0.010249116	0.016656721
Balanced	B8	L24_E12_10_B8_D14_16	0.013395854	0.020195358
Balanced	B8	L24_E12_10_B8_D16_18	0.01126899	0.019461954
Balanced	B8	L24_E12_10_B8_D18_20	0.008945597	0.014437428
Balanced	B8	L24_E12_10_B8_D20_22	0.012319727	0.018815922
Balanced	B8	L24_E14_12_B8_D8_10	0.012731771	0.021036537
Balanced	B8	L24_E14_12_B8_D10_12	0.011132245	0.017919274
Balanced	B8	L24_E14_12_B8_D12_14	0.013684283	0.020270029
Balanced	B8	L24_E14_12_B8_D14_16	0.010775772	0.017332792
Balanced	B8	L24_E14_12_B8_D16_18	0.011125657	0.017446777
Balanced	B8	L24_E14_12_B8_D18_20	0.01094774	0.01628216
Balanced	B8	L24_E14_12_B8_D20_22	0.007277598	0.014432641
Balanced	B8	L24_E16_14_B8_D8_10	0.015016627	0.022679856
Balanced	B8	L24_E16_14_B8_D10_12	0.013373789	0.019602077
Balanced	B8	L24_E16_14_B8_D12_14	0.012410827	0.019148977
Balanced	B8	L24_E16_14_B8_D14_16	0.011969252	0.017746914
Balanced	B8	L24_E16_14_B8_D16_18	0.00820365	0.013060251
Balanced	B8	L24_E16_14_B8_D18_20	0.007562756	0.01258753
Balanced	B8	L24_E16_14_B8_D20_22	0.007976608	0.013764562
Balanced	B8	L24_E18_16_B8_D8_10	0.019272298	0.027413798
Balanced	B8	L24_E18_16_B8_D10_12	0.018363694	0.027087046
Balanced	B8	L24_E18_16_B8_D12_14	0.011286693	0.018138178
Balanced	B8	L24_E18_16_B8_D14_16	0.010196437	0.016214075
Balanced	B8	L24_E18_16_B8_D16_18	0.009142173	0.01462759
Balanced	B8	L24_E18_16_B8_D18_20	0.007723502	0.0126371
Balanced	B8	L24_E18_16_B8_D20_22	0.016000131	0.031696063
Balanced	B8	L24_E20_18_B8_D8_10	0.012469248	0.018660728
Balanced	B8	L24_E20_18_B8_D10_12	0.011384247	0.017297527
Balanced	B8	L24_E20_18_B8_D12_14	0.007756393	0.01389248
Balanced	B8	L24_E20_18_B8_D14_16	0.010571545	0.017923929
Balanced	B8	L24_E20_18_B8_D16_18	0.010552365	0.018388778
Balanced	B8	L24_E20_18_B8_D18_20	0.011967959	0.018513737
Balanced	B8	L24_E20_18_B8_D20_22	0.008885314	0.014631391
Balanced	B8	L24_E22_20_B8_D8_10	0.015228407	0.026589355
Balanced	B8	L24_E22_20_B8_D10_12	0.010015349	0.016706284
Balanced	B8	L24_E22_20_B8_D12_14	0.010139124	0.015177434
Balanced	B8	L24_E22_20_B8_D14_16	0.007095151	0.011576039
Balanced	B8	L24_E22_20_B8_D16_18	0.0095194	0.016322633
Balanced	B8	L24_E22_20_B8_D18_20	0.008468733	0.014596486
Balanced	B8	L24_E22_20_B8_D20_22	0.00928088	0.01695844
Balanced	B10	L24_E10_8_B10_D8_10	0.020233354	0.030165875
Balanced	B10	L24_E10_8_B10_D10_12	0.011752107	0.019158585
Balanced	B10	L24_E10_8_B10_D12_14	0.013033877	0.021106085
Balanced	B10	L24_E10_8_B10_D14_16	0.009600295	0.017440444
Balanced	B10	L24_E10_8_B10_D16_18	0.012266792	0.019465182
Balanced	B10	L24_E10_8_B10_D18_20	0.012120275	0.019972069
Balanced	B10	L24_E10_8_B10_D20_22	0.011167606	0.018437615
Balanced	B10	L24_E12_10_B10_D8_10	0.016640469	0.027864916
Balanced	B10	L24_E12_10_B10_D10_12	0.014428391	0.021437526
Balanced	B10	L24_E12_10_B10_D12_14	0.011593088	0.017778683
Balanced	B10	L24_E12_10_B10_D14_16	0.01202246	0.018209089
Balanced	B10	L24_E12_10_B10_D16_18	0.011851581	0.019077662
Balanced	B10	L24_E12_10_B10_D18_20	0.012331415	0.018981433
Balanced	B10	L24_E12_10_B10_D20_22	0.013418073	0.020311581
Balanced	B10	L24_E14_12_B10_D8_10	0.013110242	0.018374674
Balanced	B10	L24_E14_12_B10_D10_12	0.011942232	0.01844489
Balanced	B10	L24_E14_12_B10_D12_14	0.012435388	0.019559812
Balanced	B10	L24_E14_12_B10_D14_16	0.011206926	0.019238029
Balanced	B10	L24_E14_12_B10_D16_18	0.011803836	0.018064195
Balanced	B10	L24_E14_12_B10_D18_20	0.007719166	0.014410676
Balanced	B10	L24_E14_12_B10_D20_22	0.014435138	0.023483554
Balanced	B10	L24_E16_14_B10_D8_10	0.011077491	0.018000929
Balanced	B10	L24_E16_14_B10_D10_12	0.01378517	0.023309732
Balanced	B10	L24_E16_14_B10_D12_14	0.010775991	0.017542208
Balanced	B10	L24_E16_14_B10_D14_16	0.008424668	0.014197184
Balanced	B10	L24_E16_14_B10_D16_18	0.00744313	0.01423794
Balanced	B10	L24_E16_14_B10_D18_20	0.008934898	0.015064557
Balanced	B10	L24_E16_14_B10_D20_22	0.007145206	0.013409941
Balanced	B10	L24_E18_16_B10_D8_10	0.017306797	0.02824511
Balanced	B10	L24_E18_16_B10_D10_12	0.010003665	0.015330164
Balanced	B10	L24_E18_16_B10_D12_14	0.009937397	0.016804602
Balanced	B10	L24_E18_16_B10_D14_16	0.012277911	0.023712898
Balanced	B10	L24_E18_16_B10_D16_18	0.011523508	0.018302547
Balanced	B10	L24_E18_16_B10_D18_20	0.012922266	0.019010309
Balanced	B10	L24_E18_16_B10_D20_22	0.009757383	0.016663309
Balanced	B10	L24_E20_18_B10_D8_10	0.010633713	0.01707088
Balanced	B10	L24_E20_18_B10_D10_12	0.012084598	0.019233264
Balanced	B10	L24_E20_18_B10_D12_14	0.012043674	0.018626062
Balanced	B10	L24_E20_18_B10_D14_16	0.00720043	0.013535816
Balanced	B10	L24_E20_18_B10_D16_18	0.008152522	0.013586865
Balanced	B10	L24_E20_18_B10_D18_20	0.006461348	0.012377882
Balanced	B10	L24_E20_18_B10_D20_22	0.007735878	0.013843983
Balanced	B10	L24_E22_20_B10_D8_10	0.013361105	0.020751674
Balanced	B10	L24_E22_20_B10_D10_12	0.008585029	0.014543698
Balanced	B10	L24_E22_20_B10_D12_14	0.010446116	0.017029785
Balanced	B10	L24_E22_20_B10_D14_16	0.008400316	0.014614398
Balanced	B10	L24_E22_20_B10_D16_18	0.006910736	0.013537439
Balanced	B10	L24_E22_20_B10_D18_20	0.008596794	0.014720659
Balanced	B10	L24_E22_20_B10_D20_22	0.008952353	0.015115184
Balanced	B12	L24_E10_8_B12_D8_10	0.01414791	0.021388927
Balanced	B12	L24_E10_8_B12_D10_12	0.015127592	0.021743722
Balanced	B12	L24_E10_8_B12_D12_14	0.016536463	0.024836285
Balanced	B12	L24_E10_8_B12_D14_16	0.012195566	0.018959139
Balanced	B12	L24_E10_8_B12_D16_18	0.009245764	0.016140325
Balanced	B12	L24_E10_8_B12_D18_20	0.016895046	0.027062286
Balanced	B12	L24_E10_8_B12_D20_22	0.01102158	0.018165346
Balanced	B12	L24_E12_10_B12_D8_10	0.01323293	0.022367517
Balanced	B12	L24_E12_10_B12_D10_12	0.012452502	0.019168895
Balanced	B12	L24_E12_10_B12_D12_14	0.009999213	0.017334832
Balanced	B12	L24_E12_10_B12_D14_16	0.012689082	0.02202631
Balanced	B12	L24_E12_10_B12_D16_18	0.010835116	0.018292062
Balanced	B12	L24_E12_10_B12_D18_20	0.008599921	0.015628215
Balanced	B12	L24_E12_10_B12_D20_22	0.014492693	0.02245375
Balanced	B12	L24_E14_12_B12_D8_10	0.012676724	0.01916324
Balanced	B12	L24_E14_12_B12_D10_12	0.008732004	0.013772049
Balanced	B12	L24_E14_12_B12_D12_14	0.011853893	0.018004227
Balanced	B12	L24_E14_12_B12_D14_16	0.011522301	0.019759638
Balanced	B12	L24_E14_12_B12_D16_18	0.008111686	0.012680038
Balanced	B12	L24_E14_12_B12_D18_20	0.008010929	0.013925698
Balanced	B12	L24_E14_12_B12_D20_22	0.009043431	0.015265347
Balanced	B12	L24_E16_14_B12_D8_10	0.018814253	0.028026126
Balanced	B12	L24_E16_14_B12_D10_12	0.011674452	0.019679721
Balanced	B12	L24_E16_14_B12_D12_14	0.0144563	0.020627506
Balanced	B12	L24_E16_14_B12_D14_16	0.008486791	0.014203359
Balanced	B12	L24_E16_14_B12_D16_18	0.008114704	0.014175036
Balanced	B12	L24_E16_14_B12_D18_20	0.009361364	0.016431458
Balanced	B12	L24_E16_14_B12_D20_22	0.006976177	0.013434468
Balanced	B12	L24_E18_16_B12_D8_10	0.011200269	0.01734951
Balanced	B12	L24_E18_16_B12_D10_12	0.008983563	0.012933123
Balanced	B12	L24_E18_16_B12_D12_14	0.007961662	0.013359638
Balanced	B12	L24_E18_16_B12_D14_16	0.008289513	0.013923645
Balanced	B12	L24_E18_16_B12_D16_18	0.010341416	0.018295681
Balanced	B12	L24_E18_16_B12_D18_20	0.010380821	0.018231886
Balanced	B12	L24_E18_16_B12_D20_22	0.007641385	0.013961554
Balanced	B12	L24_E20_18_B12_D8_10	0.013198731	0.022337751
Balanced	B12	L24_E20_18_B12_D10_12	0.008621255	0.01430322
Balanced	B12	L24_E20_18_B12_D12_14	0.007713827	0.013112054
Balanced	B12	L24_E20_18_B12_D14_16	0.011884554	0.018406086
Balanced	B12	L24_E20_18_B12_D16_18	0.008235321	0.014060645
Balanced	B12	L24_E20_18_B12_D18_20	0.010465188	0.017476635
Balanced	B12	L24_E20_18_B12_D20_22	0.006310659	0.013337531
Balanced	B12	L24_E22_20_B12_D8_10	0.012796138	0.020780822
Balanced	B12	L24_E22_20_B12_D10_12	0.010534339	0.016327217
Balanced	B12	L24_E22_20_B12_D12_14	0.008848218	0.013751759
Balanced	B12	L24_E22_20_B12_D14_16	0.008191936	0.014229388
Balanced	B12	L24_E22_20_B12_D16_18	0.006092409	0.01129731
Balanced	B12	L24_E22_20_B12_D18_20	0.009358768	0.015475836
Balanced	B12	L24_E22_20_B12_D20_22	0.005709786	0.011208725
Balanced	B14	L24_E10_8_B14_D8_10	0.016634461	0.025318654
Balanced	B14	L24_E10_8_B14_D10_12	0.012433126	0.020200988
Balanced	B14	L24_E10_8_B14_D12_14	0.011332768	0.018543165
Balanced	B14	L24_E10_8_B14_D14_16	0.008746	0.015377605
Balanced	B14	L24_E10_8_B14_D16_18	0.010634057	0.018462891
Balanced	B14	L24_E10_8_B14_D18_20	0.00952878	0.014417068
Balanced	B14	L24_E10_8_B14_D20_22	0.011492095	0.020474436
Balanced	B14	L24_E12_10_B14_D8_10	0.014938395	0.024738461
Balanced	B14	L24_E12_10_B14_D10_12	0.012164013	0.017890533
Balanced	B14	L24_E12_10_B14_D12_14	0.011970019	0.017954476
Balanced	B14	L24_E12_10_B14_D14_16	0.009808813	0.017162852
Balanced	B14	L24_E12_10_B14_D16_18	0.010080094	0.016826998
Balanced	B14	L24_E12_10_B14_D18_20	0.007452289	0.014155453
Balanced	B14	L24_E12_10_B14_D20_22	0.012036892	0.017635632
Balanced	B14	L24_E14_12_B14_D8_10	0.01119254	0.017975081
Balanced	B14	L24_E14_12_B14_D10_12	0.011954973	0.018746814
Balanced	B14	L24_E14_12_B14_D12_14	0.010655029	0.017101269
Balanced	B14	L24_E14_12_B14_D14_16	0.009596703	0.017062962
Balanced	B14	L24_E14_12_B14_D16_18	0.016678691	0.02641063
Balanced	B14	L24_E14_12_B14_D18_20	0.006756256	0.011424747
Balanced	B14	L24_E14_12_B14_D20_22	0.010530442	0.018963279
Balanced	B14	L24_E16_14_B14_D8_10	0.01080025	0.016407503
Balanced	B14	L24_E16_14_B14_D10_12	0.009111326	0.014399161
Balanced	B14	L24_E16_14_B14_D12_14	0.012275641	0.019249337
Balanced	B14	L24_E16_14_B14_D14_16	0.009489594	0.016485734
Balanced	B14	L24_E16_14_B14_D16_18	0.010920933	0.017181747
Balanced	B14	L24_E16_14_B14_D18_20	0.010095621	0.019125881
Balanced	B14	L24_E16_14_B14_D20_22	0.006382407	0.013860118
Balanced	B14	L24_E18_16_B14_D8_10	0.012584542	0.019625643
Balanced	B14	L24_E18_16_B14_D10_12	0.011038356	0.017034989
Balanced	B14	L24_E18_16_B14_D12_14	0.012321697	0.020131405
Balanced	B14	L24_E18_16_B14_D14_16	0.010447945	0.018016152
Balanced	B14	L24_E18_16_B14_D16_18	0.011566822	0.01698165
Balanced	B14	L24_E18_16_B14_D18_20	0.008635782	0.015291593
Balanced	B14	L24_E18_16_B14_D20_22	0.006776017	0.015732039
Balanced	B14	L24_E20_18_B14_D8_10	0.011501783	0.018860975
Balanced	B14	L24_E20_18_B14_D10_12	0.010963756	0.017627282
Balanced	B14	L24_E20_18_B14_D12_14	0.008121886	0.012793698
Balanced	B14	L24_E20_18_B14_D14_16	0.01155027	0.01921783
Balanced	B14	L24_E20_18_B14_D16_18	0.007514947	0.014350478
Balanced	B14	L24_E20_18_B14_D18_20	0.007838466	0.012812638
Balanced	B14	L24_E20_18_B14_D20_22	0.005615217	0.011539103
Balanced	B14	L24_E22_20_B14_D8_10	0.012641335	0.020229978
Balanced	B14	L24_E22_20_B14_D10_12	0.01176616	0.018031228
Balanced	B14	L24_E22_20_B14_D12_14	0.011317902	0.017977262
Balanced	B14	L24_E22_20_B14_D14_16	0.007932813	0.014028862
Balanced	B14	L24_E22_20_B14_D16_18	0.006903451	0.012411846
Balanced	B14	L24_E22_20_B14_D18_20	0.007345156	0.012912999
Balanced	B14	L24_E22_20_B14_D20_22	0.006574532	0.011853616
Balanced	B16	L24_E10_8_B16_D8_10	0.01443889	0.022198021
Balanced	B16	L24_E10_8_B16_D10_12	0.012854735	0.021357154
Balanced	B16	L24_E10_8_B16_D12_14	0.010929574	0.018578915
Balanced	B16	L24_E10_8_B16_D14_16	0.012646293	0.018443601
Balanced	B16	L24_E10_8_B16_D16_18	0.010945445	0.016249189
Balanced	B16	L24_E10_8_B16_D18_20	0.01253196	0.019771604
Balanced	B16	L24_E10_8_B16_D20_22	0.010982584	0.018833345
Balanced	B16	L24_E12_10_B16_D8_10	0.010218883	0.017238794
Balanced	B16	L24_E12_10_B16_D10_12	0.011635716	0.018228777
Balanced	B16	L24_E12_10_B16_D12_14	0.011159241	0.018570641
Balanced	B16	L24_E12_10_B16_D14_16	0.011635184	0.019652108
Balanced	B16	L24_E12_10_B16_D16_18	0.007601895	0.013697112
Balanced	B16	L24_E12_10_B16_D18_20	0.011896968	0.019023377
Balanced	B16	L24_E12_10_B16_D20_22	0.007902916	0.013832669
Balanced	B16	L24_E14_12_B16_D8_10	0.012019927	0.017690079
Balanced	B16	L24_E14_12_B16_D10_12	0.013313841	0.019373478
Balanced	B16	L24_E14_12_B16_D12_14	0.014628788	0.021947896
Balanced	B16	L24_E14_12_B16_D14_16	0.01148012	0.020053944
Balanced	B16	L24_E14_12_B16_D16_18	0.011231527	0.017585358
Balanced	B16	L24_E14_12_B16_D18_20	0.008087044	0.013754104
Balanced	B16	L24_E14_12_B16_D20_22	0.010989663	0.018836258
Balanced	B16	L24_E16_14_B16_D8_10	0.013403513	0.02059768
Balanced	B16	L24_E16_14_B16_D10_12	0.007176968	0.012844114
Balanced	B16	L24_E16_14_B16_D12_14	0.01657103	0.024360161
Balanced	B16	L24_E16_14_B16_D14_16	0.009002742	0.015618296
Balanced	B16	L24_E16_14_B16_D16_18	0.01029524	0.016195424
Balanced	B16	L24_E16_14_B16_D18_20	0.007956126	0.014988977
Balanced	B16	L24_E16_14_B16_D20_22	0.009384157	0.014599309
Balanced	B16	L24_E18_16_B16_D8_10	0.009789838	0.014376386
Balanced	B16	L24_E18_16_B16_D10_12	0.011615248	0.018652843
Balanced	B16	L24_E18_16_B16_D12_14	0.00925892	0.014856027
Balanced	B16	L24_E18_16_B16_D14_16	0.006569106	0.011911848
Balanced	B16	L24_E18_16_B16_D16_18	0.007048017	0.015030492
Balanced	B16	L24_E18_16_B16_D18_20	0.008414931	0.012840476
Balanced	B16	L24_E18_16_B16_D20_22	0.007079652	0.012801006
Balanced	B16	L24_E20_18_B16_D8_10	0.011535152	0.018558541
Balanced	B16	L24_E20_18_B16_D10_12	0.010705016	0.017237978
Balanced	B16	L24_E20_18_B16_D12_14	0.007287473	0.011645617
Balanced	B16	L24_E20_18_B16_D14_16	0.007791022	0.013920991
Balanced	B16	L24_E20_18_B16_D16_18	0.010161802	0.015694067
Balanced	B16	L24_E20_18_B16_D18_20	0.009407548	0.015331412
Balanced	B16	L24_E20_18_B16_D20_22	0.007467068	0.013502462
Balanced	B16	L24_E22_20_B16_D8_10	0.012481581	0.018508103
Balanced	B16	L24_E22_20_B16_D10_12	0.01088657	0.016404072
Balanced	B16	L24_E22_20_B16_D12_14	0.012155175	0.018602479
Balanced	B16	L24_E22_20_B16_D14_16	0.007059867	0.012669311
Balanced	B16	L24_E22_20_B16_D16_18	0.009708018	0.017470954
Balanced	B16	L24_E22_20_B16_D18_20	0.008027255	0.012676523
Balanced	B16	L24_E22_20_B16_D20_22	0.007021849	0.012291384
Balanced	B18	L24_E10_8_B18_D10_12	0.01695957	0.024558404
Balanced	B18	L24_E16_14_B18_D8_10	0.015121065	0.023229439
Balanced	B18	L24_E10_8_B18_D8_10	0.014125844	0.021659775
Balanced	B18	L24_E22_20_B18_D8_10	0.013319398	0.023895433
Balanced	B18	L24_E18_16_B18_D10_12	0.013304006	0.019484282
Balanced	B18	L24_E10_8_B18_D12_14	0.012314063	0.018517887
Balanced	B18	L24_E12_10_B18_D8_10	0.01228983	0.019133694
Balanced	B18	L24_E14_12_B18_D8_10	0.012207462	0.01828197
Balanced	B18	L24_E16_14_B18_D10_12	0.012191797	0.018332638
Balanced	B18	L24_E10_8_B18_D14_16	0.012113245	0.019373765
Balanced	B18	L24_E22_20_B18_D12_14	0.011712676	0.019212885
Balanced	B18	L24_E14_12_B18_D20_22	0.011445946	0.017618913
Balanced	B18	L24_E12_10_B18_D14_16	0.011347599	0.018519077
Balanced	B18	L24_E12_10_B18_D12_14	0.011273368	0.018799953
Balanced	B18	L24_E20_18_B18_D8_10	0.011256037	0.017441269
Balanced	B18	L24_E12_10_B18_D16_18	0.011087871	0.018308533
Balanced	B18	L24_E16_14_B18_D16_18	0.010889485	0.018337818
Balanced	B18	L24_E18_16_B18_D8_10	0.010852436	0.017056787
Balanced	B18	L24_E14_12_B18_D12_14	0.010757718	0.017448986
Balanced	B18	L24_E10_8_B18_D18_20	0.01065963	0.016878871
Balanced	B18	L24_E20_18_B18_D12_14	0.010095704	0.015183505
Balanced	B18	L24_E12_10_B18_D10_12	0.009935365	0.015215641
Balanced	B18	L24_E20_18_B18_D14_16	0.009471943	0.013873239
Balanced	B18	L24_E14_12_B18_D18_20	0.009420007	0.014936462
Balanced	B18	L24_E20_18_B18_D20_22	0.009338461	0.01569205
Balanced	B18	L24_E10_8_B18_D16_18	0.009127922	0.014893274
Balanced	B18	L24_E14_12_B18_D16_18	0.009079733	0.013138008
Balanced	B18	L24_E22_20_B18_D10_12	0.008860467	0.015600192
Balanced	B18	L24_E20_18_B18_D16_18	0.008754261	0.014281114
Balanced	B18	L24_E18_16_B18_D14_16	0.008641821	0.015084272
Balanced	B18	L24_E16_14_B18_D14_16	0.008624752	0.014314129
Balanced	B18	L24_E10_8_B18_D20_22	0.008519571	0.014369097
Balanced	B18	L24_E18_16_B18_D12_14	0.008169321	0.012955836
Balanced	B18	L24_E14_12_B18_D10_12	0.008127244	0.014743481
Balanced	B18	L24_E18_16_B18_D16_18	0.008117163	0.014629924
Balanced	B18	L24_E20_18_B18_D10_12	0.008108931	0.012854083
Balanced	B18	L24_E16_14_B18_D18_20	0.008108213	0.013673033
Balanced	B18	L24_E22_20_B18_D14_16	0.008028419	0.013705629
Balanced	B18	L24_E14_12_B18_D14_16	0.007903464	0.01299998
Balanced	B18	L24_E20_18_B18_D18_20	0.007793906	0.013070653
Balanced	B18	L24_E22_20_B18_D16_18	0.007581891	0.012241531
Balanced	B18	L24_E16_14_B18_D12_14	0.007469062	0.012449068
Balanced	B18	L24_E18_16_B18_D18_20	0.007463417	0.012717633
Balanced	B18	L24_E16_14_B18_D20_22	0.00730257	0.01348797
Balanced	B18	L24_E22_20_B18_D18_20	0.007228242	0.012477516
Balanced	B18	L24_E22_20_B18_D20_22	0.006432029	0.012578507
Balanced	B18	L24_E12_10_B18_D18_20	0.006421868	0.012517379
Balanced	B18	L24_E12_10_B18_D20_22	0.006370554	0.013798644
Balanced	B18	L24_E18_16_B18_D20_22	0.00622308	0.011526386
'''